## Desafio Latam - SQL

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import sqlite3


In [2]:
conn = sqlite3.connect("desafio_sql_final.db")
cursor = conn.cursor()

#### Creación de tablas 

In [3]:
cursor.execute('''
CREATE TABLE peliculas (
    id_pelicula INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL
)
''') 

In [4]:
cursor.execute('''
CREATE TABLE tag (
    id_tag INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL
)               
''')

In [5]:
cursor.execute(''' 
CREATE TABLE pelicula_tag (
    id_pelicula INTEGER,
    id_tag INTEGER,
    FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula),
    FOREIGN KEY (id_tag) REFERENCES tags(id_tags)           
)
''')

#### Poblar con datos

In [7]:
cursor.executemany('INSERT INTO peliculas (titulo) VALUES (?)', [
    ('Matrix',),
    ('Inception',),
    ('Interstellar',),
    ('Gladiator',),
    ('The Lord of the Rings',)
])

conn.commit()

In [8]:
cursor.executemany('INSERT INTO tag (nombre) VALUES (?)', [
    ('Ciencia ficcion',),
    ('Suspenso',),
    ('Accion',),
    ('Drama',),
    ('Fantasia',)
    ])

conn.commit()

In [9]:

# Ver tablas
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

# Ver datos de una tabla
pd.read_sql_query("SELECT * FROM peliculas", conn)

,id_pelicula,titulo
0,1,Matrix
1,2,Inception
2,3,Interstellar
3,4,Gladiator
4,5,The Lord of the Rings
5,6,Matrix
6,7,Inception
7,8,Interstellar
8,9,Gladiator
9,10,The Lord of the Rings


In [10]:
cursor.executemany('''
INSERT INTO pelicula_tag (id_pelicula, id_tag) 
values (?,?)
''', [
    (1,1),
    (1,2),
    (2,1),
    (2,5),
    (3,1),
    (3,2),
    (3,4),
])
conn.commit()

#### Revisar si se realizo bien el cruce entre tablas y su correcto etiquetado según el género


In [15]:
cursor.execute('''
SELECT p.titulo AS Pelicula, t.nombre AS Genero
FROM peliculas p
JOIN pelicula_tag pt ON p.id_pelicula = pt.id_pelicula
JOIN tag t ON t.id_tag = pt.id_tag
ORDER BY p.titulo
''')

for row in cursor.fetchall():
    print(row)



('Inception', 'Ciencia ficcion')
('Inception', 'Fantasia')
('Interstellar', 'Ciencia ficcion')
('Interstellar', 'Suspenso')
('Interstellar', 'Drama')
('Matrix', 'Ciencia ficcion')
('Matrix', 'Suspenso')


---

### Creación de tablas para comentarios de usuarios y revisar si su respuesta es correcta

In [31]:
cursor.execute('''
CREATE TABLE usuario (
    id_usuario INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre TEXT NOT NULL,
    edad INTEGER NOT NULL CHECK (edad >= 18),
    email TEXT UNIQUE
)
''')


In [32]:
cursor.execute('''
CREATE TABLE pregunta (
    id_pregunta INTEGER PRIMARY KEY AUTOINCREMENT,
    pregunta TEXT NOT NULL
)
''')


In [33]:
cursor.execute('''
CREATE TABLE respuesta (
    id_respuesta INTEGER PRIMARY KEY AUTOINCREMENT,
    id_usuario INTEGER NOT NULL,
    id_pregunta INTEGER NOT NULL,
    respuesta TEXT NOT NULL,
    FOREIGN KEY (id_usuario) REFERENCES usuario(id_usuario) ON DELETE CASCADE,
    FOREIGN KEY (id_pregunta) REFERENCES pregunta(id_pregunta)
)
''')


In [34]:
cursor.executemany('''
INSERT INTO usuario (nombre, edad)
VALUES (?, ?)
''', [
    ('Ana', 22),
    ('Bruno', 30),
    ('Camila', 19),
    ('Daniel', 27),
    ('Eva', 35)
])


In [35]:
cursor.executemany('''
INSERT INTO pregunta (pregunta)
VALUES (?)
''', [
    ('¿Capital de Francia?',),
    ('¿Resultado de 2+2?',),
    ('¿Color del cielo?',),
    ('¿Animal que ladra?',),
    ('¿Número de patas de una araña?',)
])


In [37]:
cursor.executemany('''
INSERT INTO respuesta (id_usuario, id_pregunta, respuesta)
VALUES (?, ?, ?)
''', [
    (1, 1, '¿Capital de Francia?'),   
    (2, 1, '¿Capital de Francia?'),   
    (3, 2, '¿Resultado de 2+2?'),      
    (4, 3, '¿Color del cielo?'),      
    (5, 3, '¿Color del cielo?'),      
    (4, 4, '¿Animal que ladra?'),       
    (5, 5, '¿Número de patas de una araña?')  
])
conn.commit()



In [38]:
query = '''
SELECT u.nombre AS Usuario, COUNT(*) AS Respuestas_Correctas
FROM usuario u
JOIN respuesta r ON u.id_usuario = r.id_usuario
JOIN pregunta p ON r.id_pregunta = p.id_pregunta
WHERE r.respuesta = p.pregunta
GROUP BY u.nombre
ORDER BY u.nombre
'''

df = pd.read_sql_query(query, conn)
df


,Usuario,Respuestas_Correctas
0,Ana,2
1,Bruno,2
2,Camila,2
3,Daniel,4
4,Eva,4


In [39]:
#borrar usuario 1
cursor.execute('''
DELETE FROM usuario
WHERE id_usuario = 1
''')
conn.commit()

In [40]:

query = '''
SELECT r.id_respuesta, u.nombre AS Usuario, p.pregunta, r.respuesta
FROM respuesta r
JOIN usuario u ON r.id_usuario = u.id_usuario
JOIN pregunta p ON r.id_pregunta = p.id_pregunta
ORDER BY r.id_respuesta
'''

df_respuestas = pd.read_sql_query(query, conn)
df_respuestas


,id_respuesta,Usuario,pregunta,respuesta
0,2,Bruno,¿Capital de Francia?,¿Capital de Francia?
1,3,Camila,¿Resultado de 2+2?,¿Resultado de 2+2?
2,4,Daniel,¿Color del cielo?,¿Color del cielo?
3,5,Eva,¿Color del cielo?,¿Color del cielo?
4,6,Daniel,¿Animal que ladra?,¿Animal que ladra?
5,7,Eva,¿Número de patas de una araña?,¿Número de patas de una araña?
6,9,Bruno,¿Capital de Francia?,¿Capital de Francia?
7,10,Camila,¿Resultado de 2+2?,¿Resultado de 2+2?
8,11,Daniel,¿Color del cielo?,¿Color del cielo?
9,12,Eva,¿Color del cielo?,¿Color del cielo?


In [41]:
cursor.execute('''
UPDATE usuario
SET email = 'ana@example.com'
WHERE nombre = 'Ana'
''')

cursor.execute('''
UPDATE usuario
SET email = 'bruno@example.com'
WHERE nombre = 'Bruno'
''')

cursor.execute('''
UPDATE usuario
SET email = 'camila@example.com'
WHERE nombre = 'Camila'
''')

cursor.execute('''
UPDATE usuario
SET email = 'daniel@example.com'
WHERE nombre = 'Daniel'
''')

cursor.execute('''
UPDATE usuario
SET email = 'eva@example.com'
WHERE nombre = 'Eva'
''')

conn.commit()




In [42]:
cursor.execute('SELECT * FROM usuario')
rows = cursor.fetchall()

for row in rows:
    print(row)


(2, 'Bruno', 30, 'bruno@example.com')
(3, 'Camila', 19, 'camila@example.com')
(4, 'Daniel', 27, 'daniel@example.com')
(5, 'Eva', 35, 'eva@example.com')


In [43]:

df_usuario = pd.read_sql_query('SELECT * FROM usuario', conn)
df_usuario


,id_usuario,nombre,edad,email
0,2,Bruno,30,bruno@example.com
1,3,Camila,19,camila@example.com
2,4,Daniel,27,daniel@example.com
3,5,Eva,35,eva@example.com
